In [ ]:
%pip install statsmodels --quiet
%pip install powerlaw --quiet

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from statsmodels.formula.api import ols 
import math
import powerlaw 
import scipy 
import random 

In [ ]:
# loading in data 
G = nx.read_edgelist("../data/loc-brightkite_edges.txt.gz")
G.nodes

In [ ]:
class SIRModelBase:
    def __init__(self, G) -> None:
        self.G = G
        self.t = 0
        self.infected = set()
        self.susceptible = set(self.G.nodes)
        self.recovered = set()
    
    def iterate(self, n: int):
        n = 1 if n is None else n
        results = []
        for _ in range(n):
            new_state, newly_infected, newly_recovered = self._iterate()
            results.append((new_state, newly_infected, newly_recovered))
        if n == 1:
            return results[0]
        else:
            return results
    
    def try_recover(self, node):
        return random.random() < self.gamma

class CascadeModel(SIRModelBase):
    def __init__(self, G) -> None:
        super().__init__(G)
    
    def set_initial_infected(self, initial_infected: list):
        """Set the initial infected nodes. initial_infected is a list of node ids as STRINGS."""
        self.infected = set(initial_infected)
        self.susceptible = set(self.G.nodes) - self.infected
        self.recovered = set()
    
    def set_parameters(self, beta: float, gamma: float):
        """Set the parameters of the model. Beta is the infection rate, gamma is the recovery rate."""
        self.beta = beta
        self.gamma = gamma
    
    def _iterate(self):
        self.t += 1
        _newly_infected = set()
        _newly_recovered = set()
        for node in self.G.nodes:
            if node in self.susceptible and self.try_infect(node):
                _newly_infected.add(node)
                continue

            if node in self.infected and self.try_recover(node):
                _newly_recovered.add(node)
                continue

            if node in self.recovered:
                continue
        new_suceptible = self.susceptible - _newly_infected
        new_infected = self.infected.union(_newly_infected) - _newly_recovered
        new_recovered = self.recovered.union(_newly_recovered)
        state = {i: 0 for i in new_suceptible}
        state.update({i: 1 for i in new_infected})
        state.update({i: 2 for i in new_recovered})
        self.susceptible = new_suceptible
        self.infected = new_infected
        self.recovered = new_recovered
        return state, _newly_infected, _newly_recovered

    def try_infect(self, node):
        neighbors = set(self.G.neighbors(node))
        infected_neighbors = neighbors.intersection(self.infected)
        fraction = len(infected_neighbors) / len(neighbors)
        
        # with open('debug.txt', 'a') as f:
            # f.write(f"{node}, {len(neighbors)}, {len(self.infected)}, {len(infected_neighbors)}, {fraction}, {neighbors}, {infected_neighbors}\n")

        return fraction >= self.beta

class ThresholdModel(SIRModelBase):
    def __init__(self, G) -> None:
        super().__init__(G)
    
    def set_initial_infected(self, initial_infected: list):
        """Set the initial infected nodes. initial_infected is a list of node ids as STRINGS."""
        self.infected = set(initial_infected)
        self.susceptible = set(self.G.nodes) - self.infected
        self.recovered = set()
    
    def set_parameters(self, theta: float, gamma: float):
        """Set the parameters of the model. Theta is the infection threshold, gamma is the recovery rate."""
        self.theta = theta
        self.gamma = gamma
    
    def _iterate(self):
        self.t += 1
        _newly_infected = set()
        _newly_recovered = set()
        for node in self.G.nodes:
            if node in self.susceptible and self.try_infect(node):
                _newly_infected.add(node)
                continue

            if node in self.infected and self.try_recover(node):
                _newly_recovered.add(node)
                continue

            if node in self.recovered:
                continue
        new_suceptible = self.susceptible - _newly_infected
        new_infected = self.infected.union(_newly_infected) - _newly_recovered
        new_recovered = self.recovered.union(_newly_recovered)
        state = {i: 0 for i in new_suceptible}
        state.update({i: 1 for i in new_infected})
        state.update({i: 2 for i in new_recovered})
        self.susceptible = new_suceptible
        self.infected = new_infected
        self.recovered = new_recovered
        return state, _newly_infected, _newly_recovered

    def try_infect(self, node):
        neighbors = set(self.G.neighbors(node))
        infected_neighbors = neighbors.intersection(self.infected)
        
        return len(infected_neighbors) >= self.theta

Cascade Model

In [ ]:
model = CascadeModel(G)
beta, gamma = 0.21204, 0.1
model.set_parameters(beta, gamma)
# Get random 1% of nodes as initial infected
percentage = 0.05
initial_infected = random.sample(list(G.nodes), math.ceil(len(G.nodes) * percentage))
model.set_initial_infected(initial_infected)
results = model.iterate(200)

# Info
# Number at final step
final_sus = list(results[-1][0].values()).count(0)
final_inf = list(results[-1][0].values()).count(1)
final_rec = list(results[-1][0].values()).count(2)
print(f"Final susceptible: {final_sus}")
print(f"Final infected: {final_inf}")
print(f"Final recovered: {final_rec}")

Threshold model

In [ ]:
model = ThresholdModel(G)
theta, gamma = 4, 0.1
model.set_parameters(theta, gamma)
# Initial infected
percentage = 0.01
initial_infected = random.sample(list(G.nodes), math.ceil(len(G.nodes) * percentage))
model.set_initial_infected(initial_infected)
results = model.iterate(50)

# Info
# Number at final step
final_sus = list(results[-1][0].values()).count(0)
final_inf = list(results[-1][0].values()).count(1)
final_rec = list(results[-1][0].values()).count(2)
print(f"Final susceptible: {final_sus}")
print(f"Final infected: {final_inf}")
print(f"Final recovered: {final_rec}")

In [ ]:
# Plotting  the graph
num_susceptible = [len(G.nodes) - len(initial_infected)]
num_infected = [len(initial_infected)]
num_recovered = [0]
for state, i, r in results:
    num_susceptible.append(list(state.values()).count(0))
    num_infected.append(list(state.values()).count(1))
    num_recovered.append(list(state.values()).count(2))

num_susceptible = np.array(num_susceptible)
num_infected = np.array(num_infected)
num_recovered = np.array(num_recovered)
# print(num_susceptible, num_infected, num_recovered)

# get proportion
prop_susceptible = num_susceptible / len(G.nodes)
prop_infected = num_infected / len(G.nodes)
prop_recovered = num_recovered / len(G.nodes)

plt.plot(prop_susceptible, label="Susceptible", color="blue")
plt.plot(prop_infected, label="Infected", color="red")
plt.plot(prop_recovered, label="Recovered", color="green")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Number of nodes")
plt.show()

In [ ]:
# A function to make the simple SI step. Loop over
# all neighbors of nodes in I and transition them
# to I with probability beta. However, r nodes cannot
# be put in S state, so they are  removed from the pool.
def si(G, i_nodes, r_nodes, beta):
   new_infected = i_nodes.copy()
   s_neighbors_of_i = {n for i_node in i_nodes for n in G.neighbors(i_node)} - r_nodes
   for s_node in s_neighbors_of_i:
      if random.random() < beta:
         new_infected.add(s_node)
   return new_infected

In [ ]:
# A function to make the simple IR  step. Loop over
# all neighbors of nodes in I and transition them
# to S with probability mu.
def recovery(G, i_nodes, r_nodes, mu):
   new_infected = i_nodes.copy()
   for i_node in i_nodes:
      if random.random() < mu:
         new_infected.remove(i_node)
         r_nodes.add(i_node)
   return new_infected, r_nodes

In [ ]:
# Wrapper function
def epidemics(G, beta, mu, steps):
   # Pick the initial infected set
   i_nodes = set(random.sample(list(G.nodes), int(0.1*len(G.nodes))))
   r_nodes = set([])
   r_sizes = [0,]
   i_sizes = [len(i_nodes)/len(G.nodes)]
   # Run for 100 steps
   for _ in range(steps):
      i_nodes = si(G, i_nodes, r_nodes, beta)
      i_nodes, r_nodes = recovery(G, i_nodes, r_nodes, mu)
      r_sizes.append(len(r_nodes) / len(G.nodes))
      i_sizes.append(len(i_nodes) / len(G.nodes))
   return r_sizes, i_sizes

In [ ]:
beta = 0.2
for mu in (0.01, 0.02, 0.04, 0.25):
   print(mu)
   R,I = epidemics(G, beta, mu, 400)
   plt.plot(R)
   plt.plot(I,color="red")
   plt.xlabel("Time steps")
   plt.ylabel("Proportion of nodes")
   plt.show()
   plt.clf()